# Implementing gradient descent

### Loading and data cleanup

In [1]:
import numpy as np
import pandas as pd

In [2]:
admissions = pd.read_csv('binary.csv')
print(admissions[:3])

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1


In [3]:
# create dummies for rank
data = pd.concat( [ admissions, pd.get_dummies(admissions['rank'], prefix='rank') ], axis=1 )
data = data.drop( 'rank', axis=1 )
print(data[:3])

   admit  gre   gpa  rank_1  rank_2  rank_3  rank_4
0      0  380  3.61       0       0       1       0
1      1  660  3.67       0       0       1       0
2      1  800  4.00       1       0       0       0


In [4]:
# standardize gre and gpa
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:, field] = (data[field] - mean)/std
print(data[:3])

   admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
0      0 -1.798011  0.578348       0       0       1       0
1      1  0.625884  0.736008       0       0       1       0
2      1  1.837832  1.603135       1       0       0       0


In [5]:
# split off 10% of data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

In [6]:
# split the data into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

### Implementing gradient descent

In [7]:
# activation function
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x ))

In [8]:
n_records, n_features = features.shape
print("#records:", n_records)
print("#features:", n_features )

#records: 360
#features: 6


##### Initialize weights

In [9]:
n_features = len(features.columns)
weights = np.random.normal( scale=1/n_features, size=n_features)

##### neural network hyper parameters

In [10]:
epochs = 1000
learning_rate = 0.5

##### run GD

In [11]:
last_loss = None

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # loop through all records and update weights
        output = sigmoid(np.dot(x, weights))
        error = y - output
        del_w += error * output * (1-output) * x
    weights += learning_rate * del_w / n_records
    
    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.2494683036003106
Train loss:  0.20827954923277536
Train loss:  0.20104154309787128
Train loss:  0.1987910557398068
Train loss:  0.19789737623011755
Train loss:  0.19748222246597247
Train loss:  0.19726814466109716
Train loss:  0.1971494231932749
Train loss:  0.19708003820439637
Train loss:  0.19703788990710588


In [12]:
# calculate accuracy on testdata
test_out = sigmoid( np.dot( features_test, weights ))
predications = test_out > 0.5 # convert output to boolean (admitted or not admitted)
accuracy = np.mean( predications == targets_test ) # compare the accuracy
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.725
